<a href="https://colab.research.google.com/github/VlasMA/My_project_AB_test/blob/main/AB_test_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.
### Mobile Games AB Testing with Cookie Cats

https://drive.google.com/file/d/19IHIuYc3Zu7yGOpdPtH-cjxkDlEbQ5BO/view?usp=sharing

In [4]:
!gdown --id 1ZnNqTVn47rku_uX5CjreofP3Xrnx1TKq

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ZnNqTVn47rku_uX5CjreofP3Xrnx1TKq
To: /content/gb_sem_9_hw.csv
100% 2.71M/2.71M [00:00<00:00, 199MB/s]


In [5]:
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats import proportion
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power

from math import asin
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm_notebook
from typing import Union

In [7]:
df = pd.read_csv('/content/gb_sem_9_hw.csv')

In [9]:
df.head

<bound method NDFrame.head of         userid  version  sum_gamerounds  retention_1  retention_7
0          116  gate_30               3        False        False
1          337  gate_30              38         True        False
2          377  gate_40             165         True        False
3          483  gate_40               1        False        False
4          488  gate_40             179         True         True
...        ...      ...             ...          ...          ...
90184  9999441  gate_40              97         True        False
90185  9999479  gate_40              30        False        False
90186  9999710  gate_30              28         True        False
90187  9999768  gate_40              51         True        False
90188  9999861  gate_40              16        False        False

[90189 rows x 5 columns]>

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [11]:
df.version.replace({'gate_30': 0, 'gate_40': 1}, inplace=True)    # изменение контроль на 0 . третамент на 1
df.retention_1.replace({'True': 0, 'False': 1}, inplace=True)   
df.retention_7.replace({'True': 0, 'False': 1}, inplace=True)  

In [12]:
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,0,3,False,False
1,337,0,38,True,False
2,377,1,165,True,False
3,483,1,1,False,False
4,488,1,179,True,True
...,...,...,...,...,...
90184,9999441,1,97,True,False
90185,9999479,1,30,False,False
90186,9999710,0,28,True,False
90187,9999768,1,51,True,False


In [13]:
df.describe()  

,userid,version,sum_gamerounds
count,9.018900e+04,90189.000000,90189.000000
mean,4.998412e+06,0.504374,51.872457
std,2.883286e+06,0.499984,195.050858
min,1.160000e+02,0.000000,0.000000
25%,2.512230e+06,0.000000,5.000000
50%,4.995815e+06,1.000000,16.000000
75%,7.496452e+06,1.000000,51.000000
max,9.999861e+06,1.000000,49854.000000


In [14]:
df.userid.nunique()

90189

In [15]:
#  кол-во true
df.retention_1.sum()

40153

In [16]:
# функция проверяет распределение на нормальность
def is_normal_distr(x):
    
    if x.size < 5_000:
        _, pvalue = stats.shapiro(x)
    else:
        _, pvalue = stats.kstest(x, 'norm')
        
    return pvalue

In [17]:
# функция для долевой (пропорции). У нас исследуется RETENTION а это долевая метрика
def calc_proportion_effect(control: pd.DataFrame,
                           treatment: pd.DataFrame,
                           column: str,) -> pd.DataFrame:
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    conv1 = control.loc[:, column].sum()
    conv2 = treatment.loc[:, column].sum()
    
    nobs1 = control.shape[0]
    nobs2 = treatment.shape[0]
    
    # effect_size_f = 2  * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))
    effect_size = proportion.proportion_effectsize(prop1=conv1/nobs1, prop2=conv2/nobs2)
    
    _, chi_pvalue, _ = proportion.proportions_chisquare([conv1, conv2], [nobs1, nobs2])
    
    _, z_pvalue = proportion.proportions_ztest([conv1, conv2], [nobs1, nobs2])

    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=nobs1,
                               alpha=0.05,
                               power=None,
                               ratio=nobs2/nobs1)
    
    pw_settings = {'alpha': .05, 'power': .8}
    pw_nobs = zt_ind_solve_power(effect_size=effect_size,
                                 nobs1=None,
                                 alpha=pw_settings['alpha'],
                                 power=pw_settings['power'],
                                 ratio=1)
    
    difference = treatment_mean - control_mean
    
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha_chi': chi_pvalue, 
                           'alpha_z': z_pvalue,
                           'beta': (1-power),
                           'power': power,
                           'difference': difference,
                           'nobs': nobs1 + nobs2},
                          index=[column]) 
    
    perfect_way = pd.DataFrame({'effect_size': effect_size,
                                'alpha_chi': pw_settings['alpha'],
                                'alpha_z': pw_settings['alpha'],
                                'beta': 1 - pw_settings['power'],
                                'power': pw_settings['power'],
                                'difference': difference,
                                'nobs': round(pw_nobs * 2, 0)},
                               index=['perfect_way'])
    
    return pd.concat((result, perfect_way))

In [18]:
# оценка. разделим тесты
control = df[df.version == 0].copy()
treatment = df[df.version == 1].copy()

In [19]:
### Testing timespent
control.shape, treatment.shape

((44700, 5), (45489, 5))

In [20]:
is_normal_distr(control.retention_1), is_normal_distr(treatment.retention_1)

(0.0, 0.0)

## Из результата видно, что выборка не соответствует нормальному распределению

In [21]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=control.retention_1, name='control',))
fig.add_trace(go.Histogram(x=treatment.retention_1, name='treatment',))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [22]:
!pip install distfit -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.8 MB/s eta 0:00:00


In [23]:
# этот модуль делает все за нас и не нужно считать никаких теств, что были выше
from distfit import distfit

# Initialize for discrete distribution fitting
dfit = distfit()

# Run distfit to and determine whether we can find the parameters from the data.
result = dfit.fit_transform(treatment.retention_1)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 154.127] [loc=0.442 scale=0.497]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 137.605] [loc=0.000 scale=0.442]
[distfit] >INFO> [pareto    ] [0.17 sec] [RSS: 137.605] [loc=-16777216.000 scale=16777216.000]
[distfit] >INFO> [dweibull  ] [2.68 sec] [RSS: 164.159] [loc=0.500 scale=0.500]
[distfit] >INFO> [t         ] [5.88 sec] [RSS: 163.184] [loc=0.000 scale=0.000]
[distfit] >INFO> [genextreme] [7.96 sec] [RSS: 164.159] [loc=0.000 scale=0.000]
[distfit] >INFO> [gamma     ] [0.58 sec] [RSS: 102.865] [loc=-0.000 scale=0.296]
[distfit] >INFO> [lognorm   ] [1.32 sec] [RSS: 144.11] [loc=-0.000 scale=0.211]
[distfit] >INFO> [beta      ] [1.13 sec] [RSS: 147.643] [loc=-0.211 scale=1.211]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 146.159] [loc=0.000 scale=1.000]
[distfit] >INFO> [loggamma  ] [0.54 sec] [RSS: 154.063] [loc=-124.826 scale=17.605]
[distfit] >INFO> Compute confidence intervals [para

In [24]:
result['model']

{'name': 'gamma',
 'score': 102.86512576864338,
 'loc': -4.391437485659767e-12,
 'scale': 0.2959915605630503,
 'arg': (0.16575030156555298,),
 'params': (0.16575030156555298, -4.391437485659767e-12, 0.2959915605630503),
 'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7fc0506a1d90>,
 'bootstrap_score': 0,
 'bootstrap_pass': None,
 'color': '#e41a1c',
 'CII_min_alpha': 2.6634493645102953e-09,
 'CII_max_alpha': 0.2644922813573899}

### Гамма - непрерывное распределение

In [25]:
calc_proportion_effect(control, treatment, 'retention_1')

,effect_size,alpha_chi,alpha_z,beta,power,difference,nobs
retention_1,0.011882,0.07441,0.07441,0.569714,0.430286,-0.005905,90189.0
perfect_way,0.011882,0.05000,0.05000,0.200000,0.800000,-0.005905,222380.0


### difference показывает среднее значение 0,05 это означает, что нет разницы между тестированием. Соответсвтенно, сколько бы тестов не делали, никакой разницы мы не увидим

In [26]:
# работаем с конверсией
fig = go.Figure()
fig.add_trace(go.Histogram(x=control.retention_1, name='control',))
fig.add_trace(go.Histogram(x=treatment.retention_1, name='treatment',))

# Overlay both histograms
fig.update_layout()
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [27]:
# доказываем что разницы нет
calc_proportion_effect(control, treatment, 'retention_1')

,effect_size,alpha_chi,alpha_z,beta,power,difference,nobs
retention_1,0.011882,0.07441,0.07441,0.569714,0.430286,-0.005905,90189.0
perfect_way,0.011882,0.05000,0.05000,0.200000,0.800000,-0.005905,222380.0


## Проверим для RETENTION 7 дня

In [28]:
#  кол-во true
df.retention_7.sum()

16781

In [29]:
is_normal_distr(control.retention_7), is_normal_distr(treatment.retention_7)

(0.0, 0.0)

### из результата видно, что выборка не соответствует нормальному распределению

In [30]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=control.retention_7, name='control',))
fig.add_trace(go.Histogram(x=treatment.retention_7, name='treatment',))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [31]:
# этот модуль делает все за нас и не нужно считать никаких теств, что были выше
from distfit import distfit

# Initialize for discrete distribution fitting
dfit = distfit()

# Run distfit to and determine whether we can find the parameters from the data.
result = dfit.fit_transform(treatment.retention_7)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.01 sec] [RSS: 184.845] [loc=0.182 scale=0.386]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 118.674] [loc=0.000 scale=0.182]
[distfit] >INFO> [pareto    ] [0.14 sec] [RSS: 118.674] [loc=-26000024.462 scale=26000024.462]
[distfit] >INFO> [dweibull  ] [4.11 sec] [RSS: 114.701] [loc=0.000 scale=0.054]
[distfit] >INFO> [t         ] [6.64 sec] [RSS: 202.95] [loc=-0.000 scale=0.000]
[distfit] >INFO> [genextreme] [4.73 sec] [RSS: 202.95] [loc=0.000 scale=0.000]
[distfit] >INFO> [gamma     ] [0.62 sec] [RSS: 142.993] [loc=-0.000 scale=0.419]
[distfit] >INFO> [lognorm   ] [1.29 sec] [RSS: 202.95] [loc=-0.000 scale=0.000]
[distfit] >INFO> [beta      ] [1.18 sec] [RSS: 202.95] [loc=-0.000 scale=1.000]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 185.95] [loc=0.000 scale=1.000]
[distfit] >INFO> [loggamma  ] [0.54 sec] [RSS: 185.069] [loc=-115.316 scale=15.637]
[distfit] >INFO> Compute confidence intervals [paramet

In [32]:
result['model']

{'name': 'dweibull',
 'score': 114.70069538040342,
 'loc': 4.0953618136199e-30,
 'scale': 0.05436947392324508,
 'arg': (0.6440886424934653,),
 'params': (0.6440886424934653, 4.0953618136199e-30, 0.05436947392324508),
 'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7fc059e3b070>,
 'bootstrap_score': 0,
 'bootstrap_pass': None,
 'color': '#e41a1c',
 'CII_min_alpha': -0.1984835165836333,
 'CII_max_alpha': 0.19848351658363325}

### dweibull - это непрерывное распределение

In [33]:
calc_proportion_effect(control, treatment, 'retention_7')

,effect_size,alpha_chi,alpha_z,beta,power,difference,nobs
retention_7,0.021074,0.001554,0.001554,0.11423,0.88577,-0.008201,90189.0
perfect_way,0.021074,0.050000,0.050000,0.20000,0.80000,-0.008201,70692.0


### Результат теста положительный
### 1) Ошибки меньше, чем пороговые (0.05/0.2)
### 2) Это доказывает стат значимость эксперимента.
### 3) Знак "-" указывает на то, что контрольная версия лучше, чем тестовая.